In [156]:
import requests
from bs4 import BeautifulSoup
import os
import json
import pprint as pp

# Create a directory to saving files
file_path = r'./manpower104'
if not os.path.exists(file_path): os.mkdir(file_path)  
    
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'} 
keyword = '後端工程師'

# synonym dictionary
skill_dict = {}
skill_path = r'./manpower104/skill/skilldict.txt'
with open(skill_path,'r',encoding='utf-8-sig') as s:
    s_str = s.read().split('\n')
for each_row in s_str:
    skill_dict[each_row.split(',')[0]] = 0
# print(skill_dict)

# with open(synonym_path, 'r', encoding='utf-8') as syn:
#     syn_str = syn.read().split('\n')
# for each_row in syn_str:
#     synonym_dict[each_row.split(',')[0]] = [item for item in each_row.split(',')]


url_start = "https://www.104.com.tw/jobs/search/?ro=0&isnew=30&keyword=%s"%(keyword)
res_start = requests.get(url_start, headers=headers)

# get total pages of 104
pages=int(res_start.text.split('"totalPage":')[1].split(',"totalCount"')[0])

# how many pages need scrap.
for page in range(1):
    res_page=requests.get(url_start+"&page=%s"%(page))
    soup_page = BeautifulSoup(res_page.text, 'html.parser')
    article_page_list = soup_page.select('h2.b-tit a')

# create title && title url
    for article in article_page_list:
        article_title = article.text
        url_article = article['href'].replace('//', 'https://')
        url_article_json = 'https://www.104.com.tw/job/ajax/content/' + url_article.split('/')[-1]
        
        # set new header of referer included.
        headers_j = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
                  'referer':url_article.split('?')[0]
                    } 
        res_article = requests.get(url_article_json, headers=headers_j) 
        job_content = json.loads(res_article.text)

#         pp.pprint(job_content)
#         print(url_article)

# def information
        job_name = (job_content['data']['header']['jobName'])
        job_company = (job_content['data']['header']['custName'])
        job_location = (job_content['data']['jobDetail']['addressRegion'])
        job_salary = (job_content['data']['jobDetail']['salary'])
        job_years = (job_content['data']['condition']['workExp'])
        job_skill = (job_content['data']['jobDetail']['jobDescription']+job_content['data']['condition']['other'])
#         print(job_skill)
        
# sum information       
        job_allinfo = ''
        job_allinfo += '工作名稱: %s\n' %(job_name)
        job_allinfo += '公司名稱: %s\n'%(job_company)
        job_allinfo += 'URL: %s\n'%(url_article)
        job_allinfo += '公司地點: %s\n'%(job_location)
        job_allinfo += '工作薪水: %s\n'%(job_salary)
        job_allinfo += '工作經歷: %s\n'%(job_years)
        
#         print(type(job_skill))
#         print(job_skill)
#         print(job_allinfo)
#         print('='*50)

# count skill       
        wordlist = jieba.cut(str(job_skill).lower())
        jieba.load_userdict('./manpower104/skill/skilldict.txt')
        cutlist = [i for i in wordlist]
        for w in skill_dict:
            if w in cutlist:
                job_allinfo += '%s: 1\n'%(w)
            else:
                job_allinfo += '%s: 0\n'%(w)  
        print(job_allinfo)
        print('='*50)
    
# save file
        try:
            with open( '%s/%s.txt' %(file_path, job_company+'_'+article_title), 'w', encoding='utf-8-sig') as f:
                f.write(job_allinfo)

        except FileNotFoundError as e:
            article_title = article_title.replace('/','&&')
            
            print(len(article_page_list))
            print(article_title)
            print(url_article)
            print(e.args)
            with open( '%s/%s.txt' %(file_path, job_company+'_'+article_title), 'w', encoding='utf-8-sig') as f:
                f.write(job_allinfo)
            
        except OSError as e:
            print('==========')
            print(url_article)
            print(e.args)
            print('==========')


工作名稱: 【Clean Code 後端研發】Java助理工程師
公司名稱: 前思資訊有限公司
URL: https://www.104.com.tw/job/735xa?jobsource=hotjob_chr
公司地點: 台北市松山區
工作薪水: 月薪28,000~35,000元
工作經歷: 1年以上
python: 0
java: 1
r語言: 0
mysql: 0
mongodb: 0
nosql: 0
sql: 1
aws: 0
gcp: 0
azure: 0
linux: 1
cloud service: 0
data mining: 0
ai: 0
Machine learning: 0
deep learning: 0
Chatbot: 0
影像辨識: 0
人臉辨識: 0
ETL: 0

工作名稱: Cloud DevOps/SRE Engineer (雲端 DevOps /SRE 工程師)
公司名稱: 希格斯資訊科技有限公司
URL: https://www.104.com.tw/job/73fno?jobsource=hotjob_chr
公司地點: 新北市新店區
工作薪水: 待遇面議
工作經歷: 3年以上
python: 1
java: 0
r語言: 0
mysql: 0
mongodb: 0
nosql: 0
sql: 0
aws: 1
gcp: 1
azure: 1
linux: 1
cloud service: 0
data mining: 0
ai: 0
Machine learning: 0
deep learning: 0
Chatbot: 0
影像辨識: 0
人臉辨識: 0
ETL: 0

25
Cloud DevOps&&SRE Engineer (雲端 DevOps &&SRE 工程師)
https://www.104.com.tw/job/73fno?jobsource=hotjob_chr
(2, 'No such file or directory')
工作名稱: 前端工程師 (VueJS / ReactJS)
公司名稱: 紅鬍子數位有限公司
URL: https://www.104.com.tw/job/6owus?jobsource=hotjob_chr
公司地點: 台北市中正區
工作薪水: 月薪40,000~60,

工作名稱: LINE TAXI 後端工程師
公司名稱: 觔斗雲聯網科技股份有限公司
URL: https://www.104.com.tw/job/6lmh9?jobsource=n104bank2
公司地點: 台北市大安區
工作薪水: 待遇面議
工作經歷: 不拘
python: 0
java: 1
r語言: 0
mysql: 1
mongodb: 0
nosql: 0
sql: 0
aws: 1
gcp: 0
azure: 0
linux: 0
cloud service: 0
data mining: 0
ai: 0
Machine learning: 0
deep learning: 0
Chatbot: 0
影像辨識: 0
人臉辨識: 0
ETL: 0

工作名稱: 平台後端工程師
公司名稱: 馬來西亞商韋博科技有限公司台灣分公司
URL: https://www.104.com.tw/job/6qbl4?jobsource=n104bank2
公司地點: 新北市板橋區
工作薪水: 待遇面議
工作經歷: 2年以上
python: 0
java: 0
r語言: 0
mysql: 1
mongodb: 0
nosql: 0
sql: 0
aws: 0
gcp: 0
azure: 0
linux: 1
cloud service: 0
data mining: 0
ai: 0
Machine learning: 0
deep learning: 0
Chatbot: 0
影像辨識: 0
人臉辨識: 0
ETL: 0

工作名稱: Python 後端工程師
公司名稱: 蓋特資訊系統股份有限公司
URL: https://www.104.com.tw/job/6vdrl?jobsource=n104bank2
公司地點: 台北市中正區
工作薪水: 月薪50,000元以上
工作經歷: 2年以上
python: 1
java: 0
r語言: 0
mysql: 0
mongodb: 0
nosql: 0
sql: 1
aws: 1
gcp: 1
azure: 1
linux: 1
cloud service: 0
data mining: 0
ai: 0
Machine learning: 0
deep learning: 0
Chatbot: 0
影像辨識: 0
人臉辨識

In [162]:
#create form table
import pandas as pd
import glob

filelist = glob.glob('./manpower104/*.txt')

rowData = []

for filename in filelist:
    with open(filename, 'r', encoding='utf-8-sig') as f:
        tmplist = f.read().split('\n')[0:-1]
        rowData.append(tmplist)       
        print(rowData)

[['工作名稱: 後端工程師 Backend Engineer', '公司名稱: 健和興端子股份有限公司', 'URL: https://www.104.com.tw/job/3yum3?jobsource=n104bank2', '公司地點: 新北市汐止區', '工作薪水: 月薪40,000~60,000元', '工作經歷: 不拘', 'python: 0', 'java: 0', 'r語言: 0', 'mysql: 1', 'mongodb: 0', 'nosql: 0', 'sql: 0', 'aws: 0', 'gcp: 0', 'azure: 0', 'linux: 1', 'cloud service: 0', 'data mining: 0', 'ai: 0', 'Machine learning: 0', 'deep learning: 0', 'Chatbot: 0', '影像辨識: 0', '人臉辨識: 0', 'ETL: 0']]
[['工作名稱: 後端工程師 Backend Engineer', '公司名稱: 健和興端子股份有限公司', 'URL: https://www.104.com.tw/job/3yum3?jobsource=n104bank2', '公司地點: 新北市汐止區', '工作薪水: 月薪40,000~60,000元', '工作經歷: 不拘', 'python: 0', 'java: 0', 'r語言: 0', 'mysql: 1', 'mongodb: 0', 'nosql: 0', 'sql: 0', 'aws: 0', 'gcp: 0', 'azure: 0', 'linux: 1', 'cloud service: 0', 'data mining: 0', 'ai: 0', 'Machine learning: 0', 'deep learning: 0', 'Chatbot: 0', '影像辨識: 0', '人臉辨識: 0', 'ETL: 0'], ['工作名稱: 後端工程師', '公司名稱: 冠宇數位科技有限公司', 'URL: https://www.104.com.tw/job/6uciu?jobsource=n104bank2', '公司地點: 台中市西區', '工作薪水: 待遇面議', '工作經歷: 不

In [163]:
# create column
columns = [r.split(': ')[0] for r in rowData[0]]

# create table
df = pd.DataFrame(data=rowData, columns=columns)
print(df)

,工作名稱,公司名稱,URL,公司地點,工作薪水,工作經歷,python,java,r語言,mysql,...,linux,cloud service,data mining,ai,Machine learning,deep learning,Chatbot,影像辨識,人臉辨識,ETL
0,工作名稱: 後端工程師 Backend Engineer,公司名稱: 健和興端子股份有限公司,URL: https://www.104.com.tw/job/3yum3?jobsourc...,公司地點: 新北市汐止區,"工作薪水: 月薪40,000~60,000元",工作經歷: 不拘,python: 0,java: 0,r語言: 0,mysql: 1,...,linux: 1,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
1,工作名稱: 後端工程師,公司名稱: 冠宇數位科技有限公司,URL: https://www.104.com.tw/job/6uciu?jobsourc...,公司地點: 台中市西區,工作薪水: 待遇面議,工作經歷: 不拘,python: 0,java: 1,r語言: 0,mysql: 1,...,linux: 0,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
2,工作名稱: 【Clean Code 後端研發】Java助理工程師,公司名稱: 前思資訊有限公司,URL: https://www.104.com.tw/job/735xa?jobsourc...,公司地點: 台北市松山區,"工作薪水: 月薪28,000~35,000元",工作經歷: 1年以上,python: 0,java: 1,r語言: 0,mysql: 0,...,linux: 1,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
3,工作名稱: 【找對地方 一展長才】PHP 工程師 / 資深工程師,公司名稱: 前思資訊有限公司,URL: https://www.104.com.tw/job/72hoj?jobsourc...,公司地點: 台北市松山區,工作薪水: 待遇面議,工作經歷: 3年以上,python: 1,java: 0,r語言: 0,mysql: 0,...,linux: 1,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
4,工作名稱: PHP後端工程師,公司名稱: 台盈資訊科技有限公司,URL: https://www.104.com.tw/job/6lgdn?jobsourc...,公司地點: 台北市松山區,工作薪水: 待遇面議,工作經歷: 1年以上,python: 0,java: 0,r語言: 0,mysql: 1,...,linux: 0,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
5,工作名稱: .NET後端工程師,公司名稱: 和泰聯網股份有限公司,URL: https://www.104.com.tw/job/71f3c?jobsourc...,公司地點: 台北市中山區,"工作薪水: 月薪40,000~75,000元",工作經歷: 2年以上,python: 1,java: 0,r語言: 0,mysql: 0,...,linux: 0,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
6,工作名稱: 後端工程師 Backend Engineer,公司名稱: 塞席爾商悠福科技股份有限公司,URL: https://www.104.com.tw/job/5vv2u?jobsourc...,公司地點: 台北市中山區,工作薪水: 待遇面議,工作經歷: 不拘,python: 0,java: 0,r語言: 0,mysql: 0,...,linux: 0,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
7,工作名稱: Backend Development 後端工程師,公司名稱: 宇泰華科技股份有限公司,URL: https://www.104.com.tw/job/5bdsg?jobsourc...,公司地點: 台北市南港區,工作薪水: 待遇面議,工作經歷: 3年以上,python: 0,java: 0,r語言: 0,mysql: 1,...,linux: 1,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
8,工作名稱: 後端工程師,公司名稱: 尊承科技股份有限公司,URL: https://www.104.com.tw/job/6xs3f?jobsourc...,公司地點: 台中市西屯區,工作薪水: 待遇面議,工作經歷: 不拘,python: 0,java: 1,r語言: 0,mysql: 1,...,linux: 0,cloud service: 0,data mining: 0,ai: 0,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0
9,工作名稱: 資深後端工程師,公司名稱: 岳鼎股份有限公司,URL: https://www.104.com.tw/job/71da7?jobsourc...,公司地點: 台北市內湖區,"工作薪水: 月薪55,000~85,000元",工作經歷: 5年以上,python: 0,java: 0,r語言: 0,mysql: 0,...,linux: 0,cloud service: 0,data mining: 0,ai: 1,Machine learning: 0,deep learning: 0,Chatbot: 0,影像辨識: 0,人臉辨識: 0,ETL: 0


In [164]:
# remove title words
for c in df:
    df[c] = df[c].apply(lambda x: x.split(': ')[-1])
print(df)

,工作名稱,公司名稱,URL,公司地點,工作薪水,工作經歷,python,java,r語言,mysql,...,linux,cloud service,data mining,ai,Machine learning,deep learning,Chatbot,影像辨識,人臉辨識,ETL
0,後端工程師 Backend Engineer,健和興端子股份有限公司,https://www.104.com.tw/job/3yum3?jobsource=n10...,新北市汐止區,"月薪40,000~60,000元",不拘,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,後端工程師,冠宇數位科技有限公司,https://www.104.com.tw/job/6uciu?jobsource=n10...,台中市西區,待遇面議,不拘,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,【Clean Code 後端研發】Java助理工程師,前思資訊有限公司,https://www.104.com.tw/job/735xa?jobsource=hot...,台北市松山區,"月薪28,000~35,000元",1年以上,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,【找對地方 一展長才】PHP 工程師 / 資深工程師,前思資訊有限公司,https://www.104.com.tw/job/72hoj?jobsource=hot...,台北市松山區,待遇面議,3年以上,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,PHP後端工程師,台盈資訊科技有限公司,https://www.104.com.tw/job/6lgdn?jobsource=n10...,台北市松山區,待遇面議,1年以上,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,.NET後端工程師,和泰聯網股份有限公司,https://www.104.com.tw/job/71f3c?jobsource=n10...,台北市中山區,"月薪40,000~75,000元",2年以上,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,後端工程師 Backend Engineer,塞席爾商悠福科技股份有限公司,https://www.104.com.tw/job/5vv2u?jobsource=n10...,台北市中山區,待遇面議,不拘,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Backend Development 後端工程師,宇泰華科技股份有限公司,https://www.104.com.tw/job/5bdsg?jobsource=n10...,台北市南港區,待遇面議,3年以上,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
8,後端工程師,尊承科技股份有限公司,https://www.104.com.tw/job/6xs3f?jobsource=n10...,台中市西屯區,待遇面議,不拘,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,資深後端工程師,岳鼎股份有限公司,https://www.104.com.tw/job/71da7?jobsource=n10...,台北市內湖區,"月薪55,000~85,000元",5年以上,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [165]:
# save cvs.file
df.to_csv('./manpower104/manpower_table.csv') 
print('done!')

done!
